In [3]:
import jsonlines
import numpy as np
import os
import pandas as pd
import re
from scipy import stats
import yaml

# read the yaml files

In [2]:
data_dir = os.path.join(os.getenv("DATA_DIR"), "mica_text_coref/movie_coref/results/coreference")
data_dirs = [os.path.join(data_dir, "Nov23-25"), os.path.join(data_dir, "Nov26-27")]

In [33]:
header = ["dir", "preprocess", "bert_lr", "coref_lr", "character_lr", "schedule", "weight_decay", "dropout", "genre", "add_cr_to_coarse", "train_document_len", "cr_seq_len", "cr_hidden_size",
          "dev_score", "train_score", "epoch"]
rows = []
for dir in data_dirs:
    for subdir in os.listdir(dir):
        result_file = os.path.join(dir, subdir, "result.yaml")
        if os.path.exists(result_file):
            with open(result_file, "r") as fr:
                result = yaml.load(fr, Loader=yaml.FullLoader)
            rows.append([subdir, result["preprocess"], result["bert_lr"], result["coref_lr"], result["character_lr"], result["warmup"], result["weight_decay"], result["dropout"], 
                         result["genre"], result["add_cr_to_coarse"], result["train_document_len"], result["cr_seq_len"], result["character_recognition"]["gru_hidden_size"], result["dev_score"], 
                         result["train_score"], result["epoch"]])
df = pd.DataFrame(rows, columns=header)
print(df.shape)
print(df.dtypes)
display(df.head(5))

(303, 16)
dir                    object
preprocess             object
bert_lr               float64
coref_lr              float64
character_lr          float64
schedule              float64
weight_decay          float64
dropout               float64
genre                  object
add_cr_to_coarse         bool
train_document_len      int64
cr_seq_len              int64
cr_hidden_size          int64
dev_score             float64
train_score           float64
epoch                   int64
dtype: object


,dir,preprocess,bert_lr,coref_lr,character_lr,schedule,weight_decay,dropout,genre,add_cr_to_coarse,train_document_len,cr_seq_len,cr_hidden_size,dev_score,train_score,epoch
0,Nov24_04:54:25PM,addsays,0.00005,0.0001,0.0001,-1.0,0.0,0.3,wb,True,5120,256,256,64.817,72.647,2
1,Nov24_07:07:40PM,addsays,0.00005,0.0001,0.0001,0.0,0.0,0.3,wb,True,5120,256,256,69.277,75.100,4
2,Nov24_08:27:36AM,addsays,0.00002,0.0001,0.0001,1.0,0.0,0.0,wb,True,5120,256,256,69.687,91.127,8
3,Nov25_05:14:57AM,regular,0.00005,0.0002,0.0002,1.0,0.0,0.3,wb,True,5120,256,256,65.587,54.767,3
4,Nov24_05:19:06AM,nocharacters,0.00002,0.0001,0.0001,1.0,0.0,0.3,wb,True,5120,256,256,62.613,61.683,2


# find the best dev score hyperparams

In [34]:
df[df["dev_score"] == df["dev_score"].max()]

,dir,preprocess,bert_lr,coref_lr,character_lr,schedule,weight_decay,dropout,genre,add_cr_to_coarse,train_document_len,cr_seq_len,cr_hidden_size,dev_score,train_score,epoch
53,Nov24_11:50:00AM,regular,0.00002,0.0002,0.0002,-1.0,0.001,0.0,wb,True,5120,256,256,74.057,84.647,11
274,Nov27_03:58:46AM,regular,0.00002,0.0002,0.0002,-1.0,0.001,0.0,wb,True,5120,256,256,74.057,84.647,11
294,Nov26_09:26:33PM,regular,0.00002,0.0002,0.0002,-1.0,0.001,0.0,wb,True,5120,256,256,74.057,84.647,11
299,Nov27_01:13:03AM,regular,0.00002,0.0002,0.0002,-1.0,0.001,0.0,wb,True,5120,256,256,74.057,84.647,11


In [35]:
best_args = df[df["dev_score"] == df["dev_score"].max()].iloc[0].to_dict()
best_args.pop("dev_score")
best_args.pop("train_score")
best_args.pop("epoch")
best_args.pop("dir")
print(best_args)

{'preprocess': 'regular', 'bert_lr': 2e-05, 'coref_lr': 0.0002, 'character_lr': 0.0002, 'schedule': -1.0, 'weight_decay': 0.001, 'dropout': 0.0, 'genre': 'wb', 'add_cr_to_coarse': True, 'train_document_len': 5120, 'cr_seq_len': 256, 'cr_hidden_size': 256}


# variation

In [36]:
def vary(df, best_args, *vars, group=True):
    index = np.full((len(df),), fill_value=False, dtype=bool)
    for i, row in df.iterrows():
        index[i] = all(row[var] == best_args[var] for var in best_args.keys() if var not in vars)
    subdf = df[index]
    if group:
        _vars = sorted(best_args.keys(), key=lambda x: x in vars)
        return subdf.groupby(_vars)[["dev_score", "train_score", "epoch"]].mean()
    else:
        return subdf

# varitation w/ bert_lr

In [37]:
vary(df, best_args, "bert_lr")

dev_score  \
preprocess coref_lr character_lr schedule weight_decay dropout genre add_cr_to_coarse train_document_len cr_seq_len cr_hidden_size bert_lr              
regular    0.0002   0.0002       -1.0     0.001        0.0     wb    True             5120               256        256            0.00001    70.2870   
                                                                                                                                   0.00002    73.5922   
                                                                                                                                   0.00003    70.1500   
                                                                                                                                   0.00004    72.6500   
                                                                                                                                   0.00005    69.7200   

                                                                                                                                            train_score  \
preprocess coref_lr character_lr schedule weight_decay dropout genre add_cr_to_coarse train_document_len cr_seq_len cr_hidden_size bert_lr                
regular    0.0002   0.0002       -1.0     0.001        0.0     wb    True             5120               256        256            0.00001      72.4530   
                                                                                                                                   0.00002      84.2562   
                                                                                                                                   0.00003      75.3170   
                                                                                                                                   0.00004      84.4200   
                                                                                                                                   0.00005      79.2330   

                                                                                                                                            epoch  
preprocess coref_lr character_lr schedule weight_decay dropout genre add_cr_to_coarse train_document_len cr_seq_len cr_hidden_size bert_lr         
regular    0.0002   0.0002       -1.0     0.001        0.0     wb    True             5120               256        256            0.00001    4.0  
                                                                                                                                   0.00002   10.0  
                                                                                                                                   0.00003    4.0  
                                                                                                                                   0.00004    7.0  
                                                                                                                                   0.00005    5.0

# varitation w/ coref_lr and character_lr

In [22]:
vary(df, best_args, "coref_lr", "character_lr")

dev_score  \
preprocess bert_lr schedule weight_decay dropout genre add_cr_to_coarse train_document_len cr_seq_len cr_hidden_size coref_lr character_lr              
regular    0.00002 -1.0     0.001        0.0     wb    True             5120               256        256            0.0001   0.0001          69.5670   
                                                                                                                     0.0002   0.0002          73.5922   

                                                                                                                                            train_score  \
preprocess bert_lr schedule weight_decay dropout genre add_cr_to_coarse train_document_len cr_seq_len cr_hidden_size coref_lr character_lr                
regular    0.00002 -1.0     0.001        0.0     wb    True             5120               256        256            0.0001   0.0001            75.1570   
                                                                                                                     0.0002   0.0002            84.2562   

                                                                                                                                            epoch  
preprocess bert_lr schedule weight_decay dropout genre add_cr_to_coarse train_document_len cr_seq_len cr_hidden_size coref_lr character_lr         
regular    0.00002 -1.0     0.001        0.0     wb    True             5120               256        256            0.0001   0.0001          5.0  
                                                                                                                     0.0002   0.0002         10.0

# variation w/ schedule

In [23]:
vary(df, best_args, "schedule")

dev_score  \
preprocess bert_lr coref_lr character_lr weight_decay dropout genre add_cr_to_coarse train_document_len cr_seq_len cr_hidden_size schedule              
regular    0.00002 0.0002   0.0002       0.001        0.0     wb    True             5120               256        256            -1.0        73.5922   
                                                                                                                                   0.0        72.2500   
                                                                                                                                   1.0        71.8530   

                                                                                                                                            train_score  \
preprocess bert_lr coref_lr character_lr weight_decay dropout genre add_cr_to_coarse train_document_len cr_seq_len cr_hidden_size schedule                
regular    0.00002 0.0002   0.0002       0.001        0.0     wb    True             5120               256        256            -1.0          84.2562   
                                                                                                                                   0.0          82.8400   
                                                                                                                                   1.0          84.8630   

                                                                                                                                            epoch  
preprocess bert_lr coref_lr character_lr weight_decay dropout genre add_cr_to_coarse train_document_len cr_seq_len cr_hidden_size schedule         
regular    0.00002 0.0002   0.0002       0.001        0.0     wb    True             5120               256        256            -1.0       10.0  
                                                                                                                                   0.0        7.0  
                                                                                                                                   1.0       10.0

# variation w/ weight_decay

In [24]:
vary(df, best_args, "weight_decay")

dev_score  \
preprocess bert_lr coref_lr character_lr schedule dropout genre add_cr_to_coarse train_document_len cr_seq_len cr_hidden_size weight_decay              
regular    0.00002 0.0002   0.0002       -1.0     0.0     wb    True             5120               256        256            0.0000          70.3530   
                                                                                                                              0.0001          70.3530   
                                                                                                                              0.0010          73.5922   
                                                                                                                              0.0100          71.6800   
                                                                                                                              0.1000          73.0430   

                                                                                                                                            train_score  \
preprocess bert_lr coref_lr character_lr schedule dropout genre add_cr_to_coarse train_document_len cr_seq_len cr_hidden_size weight_decay                
regular    0.00002 0.0002   0.0002       -1.0     0.0     wb    True             5120               256        256            0.0000            71.3400   
                                                                                                                              0.0001            71.3400   
                                                                                                                              0.0010            84.2562   
                                                                                                                              0.0100            80.1800   
                                                                                                                              0.1000            84.7130   

                                                                                                                                            epoch  
preprocess bert_lr coref_lr character_lr schedule dropout genre add_cr_to_coarse train_document_len cr_seq_len cr_hidden_size weight_decay         
regular    0.00002 0.0002   0.0002       -1.0     0.0     wb    True             5120               256        256            0.0000          3.0  
                                                                                                                              0.0001          3.0  
                                                                                                                              0.0010         10.0  
                                                                                                                              0.0100          6.0  
                                                                                                                              0.1000         10.0

# variation w/ genre

In [25]:
vary(df, best_args, "genre")

dev_score  \
preprocess bert_lr coref_lr character_lr schedule weight_decay dropout add_cr_to_coarse train_document_len cr_seq_len cr_hidden_size genre              
regular    0.00002 0.0002   0.0002       -1.0     0.001        0.0     True             5120               256        256            bc       71.6430   
                                                                                                                                     bn       71.4900   
                                                                                                                                     mz       73.8600   
                                                                                                                                     nw       72.9970   
                                                                                                                                     pt       72.8530   
                                                                                                                                     tc       72.2130   
                                                                                                                                     wb       73.5922   

                                                                                                                                            train_score  \
preprocess bert_lr coref_lr character_lr schedule weight_decay dropout add_cr_to_coarse train_document_len cr_seq_len cr_hidden_size genre                
regular    0.00002 0.0002   0.0002       -1.0     0.001        0.0     True             5120               256        256            bc         75.3400   
                                                                                                                                     bn         85.4300   
                                                                                                                                     mz         85.1370   
                                                                                                                                     nw         80.9200   
                                                                                                                                     pt         82.5500   
                                                                                                                                     tc         76.5200   
                                                                                                                                     wb         84.2562   

                                                                                                                                            epoch  
preprocess bert_lr coref_lr character_lr schedule weight_decay dropout add_cr_to_coarse train_document_len cr_seq_len cr_hidden_size genre         
regular    0.00002 0.0002   0.0002       -1.0     0.001        0.0     True             5120               256        256            bc       4.0  
                                                                                                                                     bn      12.0  
                                                                                                                                     mz       9.0  
                                                                                                                                     nw       6.0  
                                                                                                                                     pt       7.0  
                                                                                                                                     tc       6.0  
                                                                                                                                     wb      10.0

In [30]:
vary(df, best_args)

,,,,,,,,,,,,dev_score,train_score,epoch
preprocess,bert_lr,coref_lr,character_lr,schedule,weight_decay,dropout,genre,add_cr_to_coarse,train_document_len,cr_seq_len,cr_hidden_size,,,
regular,0.00002,0.0002,0.0002,-1.0,0.001,0.0,wb,True,5120,256,256,73.5922,84.2562,10.0


In [38]:
vary(df, best_args, group=False)

,dir,preprocess,bert_lr,coref_lr,character_lr,schedule,weight_decay,dropout,genre,add_cr_to_coarse,train_document_len,cr_seq_len,cr_hidden_size,dev_score,train_score,epoch
53,Nov24_11:50:00AM,regular,0.00002,0.0002,0.0002,-1.0,0.001,0.0,wb,True,5120,256,256,74.057,84.647,11
274,Nov27_03:58:46AM,regular,0.00002,0.0002,0.0002,-1.0,0.001,0.0,wb,True,5120,256,256,74.057,84.647,11
283,Nov27_02:05:55AM,regular,0.00002,0.0002,0.0002,-1.0,0.001,0.0,wb,True,5120,256,256,71.733,82.693,6
294,Nov26_09:26:33PM,regular,0.00002,0.0002,0.0002,-1.0,0.001,0.0,wb,True,5120,256,256,74.057,84.647,11
299,Nov27_01:13:03AM,regular,0.00002,0.0002,0.0002,-1.0,0.001,0.0,wb,True,5120,256,256,74.057,84.647,11


In [44]:
vary(df, best_args, "cr_seq_len", "cr_hidden_size")

dev_score  \
preprocess bert_lr coref_lr character_lr schedule weight_decay dropout genre add_cr_to_coarse train_document_len cr_seq_len cr_hidden_size              
regular    0.00002 0.0002   0.0002       -1.0     0.001        0.0     wb    True             5120               256        128               70.9050   
                                                                                                                            256               73.5922   
                                                                                                                            512               71.6600   
                                                                                                                            1024              71.2915   
                                                                                                                 512        128               70.9115   
                                                                                                                            256               71.2035   
                                                                                                                            512               72.8980   
                                                                                                                            1024              72.6865   

                                                                                                                                            train_score  \
preprocess bert_lr coref_lr character_lr schedule weight_decay dropout genre add_cr_to_coarse train_document_len cr_seq_len cr_hidden_size                
regular    0.00002 0.0002   0.0002       -1.0     0.001        0.0     wb    True             5120               256        128                 78.4515   
                                                                                                                            256                 84.2562   
                                                                                                                            512                 77.4335   
                                                                                                                            1024                72.1635   
                                                                                                                 512        128                 80.4415   
                                                                                                                            256                 78.5850   
                                                                                                                            512                 82.4535   
                                                                                                                            1024                80.4870   

                                                                                                                                            epoch  
preprocess bert_lr coref_lr character_lr schedule weight_decay dropout genre add_cr_to_coarse train_document_len cr_seq_len cr_hidden_size         
regular    0.00002 0.0002   0.0002       -1.0     0.001        0.0     wb    True             5120               256        128               5.0  
                                                                                                                            256              10.0  
                                                                                                                            512               7.5  
                                                                                                                            1024              3.5  
                                                                                                                 512        128               5.5  
                      

In [45]:
round(-np.inf, 4)

-inf